In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import time

In [2]:
USE_CUDA = torch.cuda.is_available()
torch.cuda.manual_seed(1) if USE_CUDA else torch.manual_seed(1)

In [3]:
kwargs = {'num_workers': 1, 'pin_memory': True} if USE_CUDA else {}

In [4]:
import json
import loguru

with open('../configs/configs.json', 'r') as f:
    configs = json.load(f)
    loguru.logger.info('Loaded the configuration file.')

BATCH_SIZE = configs['BATCH_SIZE']
LR = configs['LR']
MOMENTUM = configs['MOMENTUM']
EPOCHS = configs['EPOCHS']
LOG_INTERVAL = configs['LOG_INTERVAL']

2024-04-04 18:40:04.055 | INFO     | __main__:<module>:6 - Loaded the configuration file.


In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs)

In [12]:
class CNNELM(nn.Module):
    def __init__(self):
        super(CNNELM, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=15, kernel_size=5, padding=1)
        self.conv2 = nn.Conv2d(in_channels=15, out_channels=210, kernel_size=4, padding=1)
        self.fc = nn.Linear(in_features=210 * 4 * 4, out_features=10, bias=False)
        loguru.logger.debug(f'params num: {sum(p.numel() for p in self.parameters() if p.requires_grad)}')
    
    def forward(self, x, to_hidden=False):
        x = self.conv1(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = F.relu(x)
        if not to_hidden:
            x = x.view(-1, self.num_flat_features(x))
            x = self.fc(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [13]:
cnnelm = CNNELM()
if USE_CUDA:
    cnnelm.cuda()

2024-04-04 19:00:52.063 | DEBUG    | __main__:__init__:7 - params num: 84600
